In [1]:
import numpy as np
import tensorflow as tf
import os

from skimage.metrics import structural_similarity as ssim   
from PIL import Image
from tensorflow import keras
from keras import layers

2025-02-13 19:04:37.661499: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-13 19:04:37.680745: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739473477.700187  198717 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739473477.705949  198717 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-13 19:04:37.725430: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
#Define the CBR (Convolutional, Batch Normalization and ReLU block)
class CBR(keras.Model):
    
    def __init__(self,filters):
        super().__init__()
        self.Convolution = layers.Conv2D(filters = filters, kernel_size = 3, strides = 1, padding = 'same', use_bias = False)
        self.BatchN = layers.BatchNormalization()
        self.relu = layers.ReLU()

    def call(self,inputs):
        x = self.Convolution(inputs)
        x = self.BatchN(x)
        return self.relu(x)    
    
#Define the Network
inputs = keras.Input(shape = (None,None,3))

CBR1 = CBR(64)(inputs)

CBR2 = CBR(64)(CBR1)

Downsampling1 = layers.MaxPooling2D(pool_size = (2,2), strides = 2)(CBR2)
CBR3 = CBR(128)(Downsampling1)

CBR4 = CBR(128)(CBR3)

Downsampling2 = layers.MaxPooling2D(pool_size = (2,2), strides = 2)(CBR4)
CBR5 = CBR(256)(Downsampling2)

CBR6 = CBR(256)(CBR5)

Downsampling3 = layers.MaxPooling2D(pool_size = (2,2), strides = 2)(CBR6)
CBR7 = CBR(512)(Downsampling3)

Upsampling1 = layers.Conv2DTranspose(filters = 256, kernel_size = 2, strides = 2, padding = 'same')(CBR7)
#Add the Skip connections
combined_images1 = layers.Concatenate(axis = -1)([Upsampling1,CBR6])
CBR8 = CBR(512)(combined_images1)

CBR9 = CBR(256)(CBR8)

Upsampling2 = layers.Conv2DTranspose(filters = 128, kernel_size = 2, strides = 2, padding = 'same')(CBR9)
combined_images2 = layers.Concatenate(axis = -1)([Upsampling2,CBR4])
CBR10 = CBR(256)(combined_images2)

CBR11 = CBR(128)(CBR10)

Upsampling3 = layers.Conv2DTranspose(filters = 64, kernel_size = 2, strides = 2, padding = 'same')(CBR11)
combined_images3 = layers.Concatenate(axis = -1)([Upsampling3,CBR2])
CBR12 = CBR(128)(combined_images3)

CBR13 = CBR(64)(CBR12)

output = layers.Conv2D(filters = 1, kernel_size = 3, strides = 1, padding = 'same')(CBR13)
Mask = keras.Model(inputs = inputs, outputs = output, name = 'Test')

2025-02-13 19:04:42.977675: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
#Load the training data
def load_dataset(top_dir = 'input_data'):
    images_dataset = []
    for name in sorted(os.listdir(top_dir)):
        img = np.array(Image.open(os.path.join(top_dir,name)))
        img = img[:,:,:3]
        images_dataset.append(img)
    return images_dataset

def load_dataset_g(top_dir = 'input_data'):
    images_dataset = []
    for name in sorted(os.listdir(top_dir)):
        img = np.array(Image.open(os.path.join(top_dir,name)))
        images_dataset.append(img)
    return images_dataset

In [4]:
rain_images_a = np.asarray(load_dataset(top_dir = '/data/lady6758/Parmeet_Datasets/NewDatasets/A_Qian_Cam_A+B/TestSet/rain_images'))
rain_images_b = np.asarray(load_dataset(top_dir = '/data/lady6758/Parmeet_Datasets/NewDatasets/B_Quan/TestSet/rain_images'))
# rain_images = np.append(rain_images_a,rain_images_b,axis = 0)
# del(rain_images_a)
# del(rain_images_b)

KwonMask_a = np.asarray(load_dataset_g(top_dir = '/data/lady6758/Parmeet_Datasets/NewDatasets/A_Qian_Cam_A+B/TestSet/KwonMask'))
KwonMask_b = np.asarray(load_dataset_g(top_dir = '/data/lady6758/Parmeet_Datasets/NewDatasets/B_Quan/TestSet/KwonMask'))
# KwonMask = np.append(KwonMask_a,KwonMask_b,axis = 0)
# del(KwonMask_a)
# del(KwonMask_b)

rain_images_a = np.float32(rain_images_a)
rain_images_b = np.float32(rain_images_b)
KwonMask_a = np.float32(KwonMask_a)
KwonMask_b = np.float32(KwonMask_b)

rain_images_a = rain_images_a/255
rain_images_b = rain_images_b/255
KwonMask_a = KwonMask_a/255
KwonMask_b = KwonMask_b/255

In [5]:
Mask.load_weights('/users/lady6758/Mayowa_4YP_Code/ReTraining_ReTesting/Raindrop_Mask/RWRetrained/attentionweights.weights.h5')

In [6]:
no_of_images = rain_images_a.shape[0]
s1 = np.empty(shape = no_of_images)
m1 = np.empty(shape = no_of_images)
for i in range(no_of_images):
    img1 = rain_images_a[i]
    img1 = np.expand_dims(img1,axis = 0)
    generated_mask = np.squeeze(np.squeeze(Mask.predict(img1),axis = -1),axis = 0)

    mask = KwonMask_a[i]
    s1[i] = ssim(mask,generated_mask,data_range = 1, gaussian_weights=True, K1 = 0.01, K2 = 0.03, use_sample_covariance = False)
    m1[i] = np.mean((mask - generated_mask)**2)
    print(s1[i])
    print(m1[i])
    print(i)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 486ms/step
0.7596384198609779
0.009384042583405972
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
0.8560600462555586
0.006979655474424362
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
0.8289902746544626
0.009887943044304848
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
0.6815138118230981
0.007237291429191828
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
0.7538251232622336
0.004972673952579498
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
0.7302330781883454
0.005344846285879612
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
0.6082920146491614
0.0031145873945206404
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
0.6354210182845682
0.0030810567550361156
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
0.6394406537799494
0.005583863705396652
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
0.6437913203826872
0.002616114914417267
9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
0.6717890755794373
0.002821563510224223
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
0.6856497672202703
0.003185237292200327
11
1/1 ━━━━━━━━━━━━━━━━━━━━

In [7]:
no_of_images = rain_images_b.shape[0]
s2 = np.empty(shape = no_of_images)
m2 = np.empty(shape = no_of_images)
for i in range(no_of_images):
    img1 = rain_images_b[i]
    img1 = np.expand_dims(img1,axis = 0)
    generated_mask = np.squeeze(np.squeeze(Mask.predict(img1),axis = -1),axis = 0)

    mask = KwonMask_b[i]
    s2[i] = ssim(mask,generated_mask,data_range = 1, gaussian_weights=True, K1 = 0.01, K2 = 0.03, use_sample_covariance = False)
    m2[i] = np.mean((mask - generated_mask)**2)
    print(s2[i])
    print(m2[i])
    print(i)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
0.6305199720185753
0.003467975649982691
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
0.6203703632792276
0.0050173308700323105
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
0.570917041373457
0.005691428203135729
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
0.6405695031308365
0.003177877049893141
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
0.6148550892806229
0.003436550498008728
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
0.5921004276544964
0.0030651954002678394
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
0.6601324753650725
0.0042302655056118965
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
0.6416693301366868
0.00501429894939065
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
0.6516335840610034
0.006264684721827507
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
0.6989102237795781
0.003918558359146118
9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
0.6627771648309734
0.0025373068638145924
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
0.6492437232427154
0.0024645363446325064
11
1/1 ━━━━━━━━━━━━━━━━━━━

In [8]:
def mean_std(array):
    mean = np.mean(array)
    std = np.std(array)
    print(f'Mean: {mean}')
    print(f'STD: {std}')

In [9]:
testa_mse = mean_std(m1)

Mean: 0.008028149136398292
STD: 0.008271941341275863


In [10]:
testa_ssim = mean_std(s1)

Mean: 0.7243641809961654
STD: 0.11989549909541351


In [11]:
testb_mse = mean_std(m2)

Mean: 0.00557203501750447
STD: 0.002822311752427947


In [12]:
testb_ssim = mean_std(s2)

Mean: 0.6430051432237571
STD: 0.1183233611109282
